In [1]:
from langchain.llms import CTransformers  # to load the llama2 model

In [2]:
llm = CTransformers(model="model\Tinyllama-150-50-200-210-80Epoch f32.gguf",
                        model_type="llama", config={'max_new_tokens':3990,'temperature':100,'context_length': 3990})

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# llm(" Please write me a SQL statement that answers the following question: How many transactions did I make this month ")

In [4]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE transactions (
	`Account_No` VARCHAR(50) NOT NULL, 
	`Transaction_details` TEXT, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_amount` INTEGER, 
	`Value_date` DATE, 
	`Date` DATE
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from transactions table:
Account_No	Transaction_details	Withdrawal_amount	Deposit_amount	Balance_amount	Value_date	Date
409000611074	TRF FROM  Indiaforensic SERVICES	0	1000000	1000000	2022-11-11	2022-11-11
409000611074	TRF FROM  Indiaforensic SERVICES	0	1000000	2000000	2022-11-17	2022-11-17
409000611074	FDRL/INTERNAL FUND TRANSFE	0	500000	2500000	2022-11-30	2022-11-30
*/


In [5]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)


# FEW SHORT LEARNING

In [6]:
#Few_shorts

few_shots=[
    {
        'Question':" Income of last 3 months. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH) """,
        'SQLResult':"1530205868",
        'Answer':"The income of last 3 months is 1530205868 ."
    }
    ,

    {
        'Question':"Total expenses/spendings of last 8 months ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH) """,
        'SQLResult':"11274528407",
        'Answer':"The total expenses of last 8 months is 11274528407 ."
    }
    ,
      {
        'Question':"Total saving of last month  ?",
        'SQLQuery':"""SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM transactions YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH); """,
        'SQLResult':"-193509",
        'Answer':"You lost -193509 last month."
    }
,
    {
        'Question':"How many transactions did I make in last 2 week  ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK) """,
        'SQLResult':"162",
        'Answer':"You made 162 transactions in last 2 week."
    },
     {
        'Question':"Amount spent last week",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK) """,
        'SQLResult':"59831261",
        'Answer':"You spent total 59831261 last week ."
    }
    , {
        'Question': "Expenses for each day this month ?",
        'SQLQuery': """SELECT Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Date """,
        'SQLResult':
        """
[(2024, 6, 2, Decimal('183648')),
(2024, 6, 3, Decimal('80666')),
(2024, 6, 4, Decimal('135504')),
(2024, 6, 5, Decimal('92031')),
(2024, 6, 6, Decimal('118475')),
(2024, 6, 7, Decimal('16440')),
(2024, 6, 9, Decimal('62112')),
(2024, 6, 10, Decimal('10331'))]
"""
        ,
        'Answer': """The total_expenses for each day of this month are:
        2024 June, 2: 183648,
        2024 June, 3: 80666,
        2024 June, 4: 135504,
        2024 June, 5: 92031,
        2024 June, 6: 118475,
        2024 June, 7: 16440,
        2024 June, 9: 62112,
        2024 June, 10: 10331
        """
    },
    {
        'Question':"savings of previous year for each month ",
        'SQLQuery':"""SELECT year(Value_date), MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY year(value_date), MONTH(Value_date) ORDER BY MONTH(Value_date); """,
        'SQLResult':"""
       [(2023, 1, Decimal('-311244')),
        (2023, 2, Decimal('9101')),
        (2023, 3, Decimal('3476')),
        (2023, 4, Decimal('-15971')),
        (2023, 5, Decimal('322249')),
        (2023, 6, Decimal('-172560')),
        (2023, 7, Decimal('361471')),
        (2023, 8, Decimal('-446844')),
        (2023, 9, Decimal('93215')),
        (2023, 10, Decimal('-73917')),
        (2023, 11, Decimal('294502')),
        (2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""
        Your Savings for each month of previous year are:
        January: -311244, February: 9101, March: 3476
        April: -15971, May: 322249, June: -172560
        July: 361471, August: -446844, September: 93215
        October: -73917, November: 294502, December: -404028 """
    },
     {
        'Question':"Amount Saved each year",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
      [(2021, Decimal('497766')),
        (2022, Decimal('272458')),
        (2023, Decimal('-340550')),
        (2024, Decimal('-148114'))]
       """,
        'Answer':"""You saved following amount each year:
                2021:497766, 2022:272458, 2023:-340550
                2024:-148114 """
    }
    ,
     {
        'Question':"Savings of last 2 year  ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date) """,
        'SQLResult':"""
       [(2022, Decimal('2270453')),
    (2023, Decimal('-895783')),
    (2024, Decimal('-912470'))]

        """,
        'Answer':"""Your savings of last 2 year are:
         2022:2270453, 2023:-895783 
                2024:-912470 ."""
    }
    ,
     {
        'Question':"Expenses of each day of last week  ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date) """,
        'SQLResult':"""
       [(2024, 6, 6, Decimal('118475')),
    (2024, 6, 7, Decimal('16440')),
    (2024, 6, 9, Decimal('62112')),
    (2024, 6, 10, Decimal('10331'))]
        """,
        'Answer':"""You spend following amount each day last week:
        June 6: 118475, June 7: 16440, June 9: 62112, June 10: 10331 """
    },
    {
        'Question':"Amount spent this year ?",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE year(Value_date) = year(curdate()) group by year(Value_date) """,
        'SQLResult':"""(2024, Decimal('6527128485'))""",
        'Answer':"You spent 6527128485 on 2024"
    },
    {
        'Question':"amount/money saved each month previous year ?",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE - INTERVAL 1 YEAR) GROUP BY YEAR(Value_date), MONTH(Value_date) """,
        'SQLResult':"""
[(2023, 1, Decimal('-311244')),
(2023, 2, Decimal('9101')),
(2023, 3, Decimal('3476')),
(2023, 4, Decimal('-15971')),
(2023, 5, Decimal('322249')),
(2023, 6, Decimal('-172560')),
(2023, 7, Decimal('361471')),
(2023, 8, Decimal('-446844')),
(2023, 9, Decimal('93215')),
(2023, 10, Decimal('-73917')),
(2023, 11, Decimal('294502')),
(2023, 12, Decimal('-404028'))]
        """,
        'Answer':"""You saved following amounts previous year :
        January: -311244, February: 9101, March: 3476, April: -15971, May: 322249, June: 172560, July: 361471, 
        August: 446844, September: 93215, October: 73917, November: 294502, December: 404028 """
    },
{
    'Question': "What is my total income last quarter?",
    'SQLQuery': """SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND QUARTER(Value_date) = QUARTER(CURRENT_DATE()) - 1""",
    'SQLResult':"2055417596",
    'Answer':"Your total income last quater was 2055417596."
}
     ,
      {
        'Question':"income of the previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"38593214103",
        'Answer':"Your income previous ysear was 38593214103."
    }
    ,
      {
        'Question':"income of the first 4 month of previous year ",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 """,
        'SQLResult':"17847505444",
        'Answer':"Your income of first 4 month of previous year was 17847505444."
    },
    {
    'Question': "amount I lost each year?",
    'SQLQuery': """ SELECT YEAR(Value_date) AS Year,SUM(Withdrawal_amount) - SUM(Deposit_amount) AS Losses FROM transactions GROUP BY YEAR(Value_date) """,
    'SQLResult': """[
        (2022, Decimal('256395842')),
        (2023, Decimal('552520828')),
        (2024, Decimal('505236609')),
        (2021, Decimal('1251859488')),
        (2020, Decimal('694854538'))]"""
    ,
    'Answer': "You lost 694854538 in 2020, 1251859488 in 2021, 256395842 in 2022, 552520828 in 2023, and -505236609 in 2024."
},
{
    "Question": "What is my average spendings of last 11 transactions?",
    "SQLQuery": "SELECT AVG(Withdrawal_amount) AS Average_Spending FROM (SELECT Withdrawal_amount FROM transactions WHERE Withdrawal_amount IS NOT NULL ORDER BY Value_date DESC LIMIT 11) AS Last11Transactions",
    "SQLResult":"1138377.5455",
    'Answer': "1138377.5455 is your average spendings of last 11 transactions ."

}
,
{
    'Question': "How much did I spend each quarter this year?",
    'SQLQuery': """SELECT QUARTER(Value_date) AS Quarter, SUM(Withdrawal_amount) AS Total_Spending
                   FROM transactions
                   WHERE YEAR(Value_date) = YEAR(CURRENT_DATE())
                   GROUP BY QUARTER(Value_date)""",
     'SQLResult':"""(1, Decimal(5885524375')),
        (2, Decimal(2689194690')),
        (3, Decimal(263256582'))""",
    'Answer': """Spendings for each quater is :
    Quarter-1: 5885524375, Quarter-2: 2689194690, Quarter-3: 263256582
         """
},
{
    'Question': "In which date did i make my highest transaction of this month and what was the amount?",
    'SQLQuery': "SELECT Value_date, MAX(Withdrawal_amount) AS Highest_Transaction_Amount FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) GROUP BY Value_date ORDER BY Highest_Transaction_Amount DESC LIMIT 1",
    'SQLResult': "(2024-06-25, Decimal('10000000'))",
    'Answer': "You made the highest transaction on Junt 25th and the amount is 10000000"
},
 {
        'Question': "How much amount did I spent on February month ?",
        'SQLQuery': """SELECT SUM(Withdrawal_amount) AS Spending_February FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = 2""",
        'SQLResult': "1985783353",
        'Answer': "You spent 1985783353 on February month."
    },
    {
        'Question': "What were my total expenditures by month in the previous year?",
        'SQLQuery': """SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Withdrawal_amount) AS Total_Expenditures FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) - 1 GROUP BY YEAR(Value_date), MONTH(Value_date)""",
        'SQLResult':"""[(2023, 1, Decimal('4530730974')), (2023, 2, Decimal('4616646063')), (2023, 3, Decimal('5276664685')), (2023, 4, Decimal('4571039249')), (2023, 5, Decimal('3557364039')), (2023, 6, Decimal('3262112460')), (2023, 7, Decimal('3142750834')), (2023, 8, Decimal('2875423837')), (2023, 9, Decimal('2727745395')), (2023, 10, Decimal('2537575257')), (2023, 11, Decimal('2818821756')), (2023, 12, Decimal('2246381750'))]""",
        'Answer': """Expenditure for each month previous year:
            January: 4530730974, February: 4616646063, March: 5276664685, April: 4571039249, May: 3557364039, June: 3262112460, July: 3142750834, August: 2875423837, September: 2727745395, October: 2537575257, November: 2818821756, December: 2246381750 """
}

,{
        'Question': "Which is my highest earning month this year ? I also want the amount along with month",
        'SQLQuery': """SELECT MONTH(Value_date) AS Month, SUM(Deposit_amount) AS Total_Earnings FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE()) GROUP BY MONTH(Value_date) ORDER BY Total_Earnings DESC limit 1""",
    'SQLResult': "[(1, Decimal('1893013978'))]",
        'Answer': "Your highest earning month this year is June with amount 1893013978."
    }

]



In [7]:
# # Define a function to transform the few_shots into the desired format
# import json
# def transform_few_shots(few_shots):
#     transformed = []
#     for shot in few_shots:
#         input_text = shot['Question']
#         output_text = shot['Answer']
#         reformatted_text = f'<s>[INST] {input_text} [/INST] {output_text} </s>'
#         transformed.append({'text': reformatted_text})
#     return transformed

# # Transform the few_shots
# transformed_few_shots = transform_few_shots(few_shots)

# # Save the transformed few_shots to a file
# with open('transformed_few_shots.json', 'w') as f:
#     json.dump(transformed_few_shots, f, indent=4)

In [8]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
to_vectorize 

[' Income of last 3 months. ? SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)  1530205868 The income of last 3 months is 1530205868 .',
 'Total expenses/spendings of last 8 months ? SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH)  11274528407 The total expenses of last 8 months is 11274528407 .',
 'Total saving of last month  ? SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM transactions YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);  -193509 You lost -193509 last month.',
 'How many transactions did I make in last 2 week   SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK)  162 You made 162 transactions in last 2 week.',
 'Amount spent last week SELECT SUM

In [10]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [11]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)
example_selector.select_examples({"Question": "Show me the breakdown of my monthly income in the previous year."})

# give me the Breakdown of income for each day this week as my account number is 409000493201'

[{'Answer': 'Your income previous ysear was 38593214103.',
  'Question': 'income of the previous year ',
  'SQLQuery': 'SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 ',
  'SQLResult': '38593214103'},
 {'Answer': 'Your income of first 4 month of previous year was 17847505444.',
  'Question': 'income of the first 4 month of previous year ',
  'SQLQuery': 'SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE MONTH(Value_date) IN (1,2,3,4) AND YEAR(Value_date) = YEAR(CURRENT_DATE) - 1 ',
  'SQLResult': '17847505444'}]

In [12]:
# # Define the custom prompt for SQL database interactions
# custom_mysql_prompt = """
# You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question. Prepare the correct MySQL query by looking at the top {top_k} similar queries.
# Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
# Never query for all columns from a table. You must query only the columns that are needed to answer the question.
# Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. 
# Pay attention to use CURDATE() function to get the current date, if the question involves "today","this year","this month" ,"this week".

# Table information:
# 1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

# Remember to only use the provided table columns and structure your queries to handle the specified requests accurately.Only answer the question given by the user and dont assume any question by yourself.
#  """

In [13]:
# # Define the custom prompt for SQL database interactions
# custom_mysql_prompt = """You are an expert in converting natural language into MySQL query.Only use the columns that are specified and needed to answer the question.
# Use query for at most {top_k} results.
# Table information:
# 1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`
# """

In [14]:
mysql_prompt = """
You are expert in converting the natural language into MYSQL query. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. 
Pay attention to use only the column names given in the below table information and never use any columns that does not exist. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today","this year","this month","this week". 
Pay attention to Emphasize using only the values present in the SQLResult to answer the question and refrain from assuming answers beyond the SQLResult.
Table information:
1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

Use the following format:

Question: Question here
SQLQuery: SQL Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [15]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt

In [16]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [17]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [18]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [19]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [20]:
suffix_eg="Question: {input}"

In [21]:
print(suffix_eg)

Question: {input}


In [22]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    # suffix="Question: {input}",
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [23]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [24]:
print(few_shot_prompt)

input_variables=['input', 'table_info', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001984E3C0290>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Only use the following tables:\n{table_info}\n\nQuestion: {input}' prefix='\nYou are expert in converting the natural language into MYSQL query. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the databas

In [25]:
# result = new_chain("What was my total earning last week as my account number is 409000493201'")

In [26]:
example_questions = [shot['Question'].strip() for shot in few_shots]
print(example_questions)

['Income of last 3 months. ?', 'Total expenses/spendings of last 8 months ?', 'Total saving of last month  ?', 'How many transactions did I make in last 2 week', 'Amount spent last week', 'Expenses for each day this month ?', 'savings of previous year for each month', 'Amount Saved each year', 'Savings of last 2 year', 'Expenses of each day of last week', 'Amount spent this year ?', 'amount/money saved each month previous year ?', 'What is my total income last quarter?', 'income of the previous year', 'income of the first 4 month of previous year', 'amount I lost each year?', 'What is my average spendings of last 11 transactions?', 'How much did I spend each quarter this year?', 'In which date did i make my highest transaction of this month and what was the amount?', 'How much amount did I spent on February month ?', 'What were my total expenditures by month in the previous year?', 'Which is my highest earning month this year ? I also want the amount along with month']


In [27]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# sentence-transformers/all-mpnet-base-v2

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
example_embeddings = model.encode(example_questions)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity  
# Function to determine if a user question is relevant to the database context
def is_question_relevant(user_question, similarity_threshold=0.40):
    user_embedding = model.encode([user_question])[0]  
    
    # calculating the cosine similarity between user question and example questions
    similarity_scores = cosine_similarity(user_embedding.reshape(1, -1), example_embeddings)[0]
    print(similarity_scores)
    
    max_similarity_score = max(similarity_scores)
    print(max_similarity_score)
    return max_similarity_score > similarity_threshold

In [30]:
from guardrails import Guard, OnFailAction
from guardrails.hub import CompetitorCheck, ToxicLanguage
from guardrails.hub import ProfanityFree

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\guardrails\validators\__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(


In [31]:

guard = Guard().use_many(
    CompetitorCheck(["khalti", "fusemachine","Deerhold","Deerwalk","Cotiviti"], on_fail=OnFailAction.EXCEPTION),
    ToxicLanguage(threshold=0.9, validation_method="sentence", on_fail=OnFailAction.EXCEPTION),
    ProfanityFree(on_fail=OnFailAction.EXCEPTION))


c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
# Function to handle user queries
def handle_user_query(user_question):
    if is_question_relevant(user_question):
        # response=new_chain(user_question)
        return True
    else:
        return "Your Question is not Relevant to the Context"

In [33]:
user_question = "What is my savings of the last 3 years ?"

In [34]:
try:
    # Validate the user question
    guard.validate(user_question)  # If validation fails, an exception will be thrown
    # If validation passes, handle the user query
    response = handle_user_query(user_question)
except Exception as e:
    response = str(e)

[0.62203085 0.5301127  0.65680283 0.29704326 0.49454123 0.41833997
 0.65930134 0.66089296 0.822416   0.3708954  0.5733789  0.6783072
 0.5384306  0.47432396 0.47007614 0.55697775 0.44578117 0.5639241
 0.2888667  0.4308848  0.55995584 0.43826628]
0.822416


In [35]:
print(response)

True


In [36]:
example_selector.select_examples({"Question":"What is my savings of the last 3 years ?"})

[{'Answer': 'Your savings of last 2 year are:\n         2022:2270453, 2023:-895783 \n                2024:-912470 .',
  'Question': 'Savings of last 2 year  ',
  'SQLQuery': 'SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date) ',
  'SQLResult': "\n       [(2022, Decimal('2270453')),\n    (2023, Decimal('-895783')),\n    (2024, Decimal('-912470'))]\n\n        "},
 {'Answer': 'You saved following amounts previous year :\n        January: -311244, February: 9101, March: 3476, April: -15971, May: 322249, June: 172560, July: 361471, \n        August: 446844, September: 93215, October: 73917, November: 294502, December: 404028 ',
  'Question': 'amount/money saved each month previous year ?',
  'SQLQuery': 'SELECT YEAR(Value_date) AS Year, MONTH(Value_date) AS Month, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings FROM transactions WHERE YE

In [37]:
if response==True:
    new_chain(f"[INST] {user_question} [/INST]")
else:
    print(response)

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
[INST] What is my savings of the last 3 years ? [/INST]
SQLQuery: